----Image_preprocessing----
Image_resize(1024, 768),
Cloth_segmentation,
Openpose,

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!unzip DCI-VTON-Virtual-Try-On.zip

In [9]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On


In [ ]:
!unzip test_dataset.zip

Densepose

In [6]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On


In [7]:
!git clone https://github.com/facebookresearch/detectron2

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [ ]:
!python -m pip install -e detectron2

In [13]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/detectron2/projects/DensePose
!pip install av>=8.0.3 opencv-python-headless>=4.5.3.56 scipy>=1.5.4

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/detectron2/projects/DensePose


In [ ]:
!python apply_net.py show configs/densepose_rcnn_R_50_FPN_s1x.yaml https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/test_dataset/test/image dp_segm -v --opts MODEL.DEVICE cpu

Parsing

In [16]:
!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.3 MB/s eta 0:00:00


In [17]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/Self-Correction-Human-Parsing/checkpoints

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/Self-Correction-Human-Parsing/checkpoints


In [18]:
!gdown 1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH

Downloading...
From (original): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=79f3d8b6-ba77-4c7e-8705-cb0c6fc225c0
To: /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/Self-Correction-Human-Parsing/checkpoints/exp-schp-201908261155-lip.pth
100% 267M/267M [00:05<00:00, 51.2MB/s]


In [19]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/Self-Correction-Human-Parsing

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/Self-Correction-Human-Parsing


In [20]:
!python3 simple_extractor.py --dataset 'lip' --model-restore 'checkpoints/exp-schp-201908261155-lip.pth' --input-dir '/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/test_dataset/test/image' --output-dir '/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/test_dataset/test/image-parse-v3'

Evaluating total class number 20 with ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat', 'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm', 'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
100% 38/38 [00:19<00:00,  1.91it/s]


Agnostic-Parsing

In [21]:
import json
from os import path as osp
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm

def get_im_parse_agnostic(im_parse, pose_data, w=768, h=1024):
    label_array = np.array(im_parse)
    parse_upper = ((label_array == 5).astype(np.float32) +
                    (label_array == 6).astype(np.float32) +
                    (label_array == 7).astype(np.float32))
    parse_neck = (label_array == 10).astype(np.float32)

    r = 10
    agnostic = im_parse.copy()

    # mask arms
    for parse_id, pose_ids in [(14, [2, 5, 6, 7]), (15, [5, 2, 3, 4])]:
        mask_arm = Image.new('L', (w, h), 'black')
        mask_arm_draw = ImageDraw.Draw(mask_arm)
        i_prev = pose_ids[0]
        for i in pose_ids[1:]:
            if (pose_data[i_prev, 0] == 0.0 and pose_data[i_prev, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
                continue
            mask_arm_draw.line([tuple(pose_data[j]) for j in [i_prev, i]], 'white', width=r*10)
            pointx, pointy = pose_data[i]
            radius = r*4 if i == pose_ids[-1] else r*15
            mask_arm_draw.ellipse((pointx-radius, pointy-radius, pointx+radius, pointy+radius), 'white', 'white')
            i_prev = i
        parse_arm = (np.array(mask_arm) / 255) * (label_array == parse_id).astype(np.float32)
        agnostic.paste(0, None, Image.fromarray(np.uint8(parse_arm * 255), 'L'))

    # mask torso & neck
    agnostic.paste(0, None, Image.fromarray(np.uint8(parse_upper * 255), 'L'))
    agnostic.paste(0, None, Image.fromarray(np.uint8(parse_neck * 255), 'L'))

    return agnostic


if __name__ =="__main__":
    data_path = '/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/test_dataset/test'
    output_path = '/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/test_dataset/test/image-parse-agnostic-v3.2'

    os.makedirs(output_path, exist_ok=True)

    for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):

        # load pose image
        pose_name = im_name.replace('.jpg', '_keypoints.json')

        try:
            with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data)
                pose_data = pose_data.reshape((-1, 3))[:, :2]
        except IndexError:
            print(pose_name)
            continue

        # load parsing image
        parse_name = im_name.replace('.jpg', '.png')
        im_parse = Image.open(osp.join(data_path, 'image-parse-v3', parse_name))

        agnostic = get_im_parse_agnostic(im_parse, pose_data)

        agnostic.save(osp.join(output_path, parse_name))

100%|██████████| 38/38 [00:03<00:00, 11.39it/s]


Warping

In [22]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/warp/test/checkpoints

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/warp/test/checkpoints


In [23]:
!gdown 1EY52bIpxMdf7YBRhkqaIx7aYx1F87E8n

Downloading...
From (original): https://drive.google.com/uc?id=1EY52bIpxMdf7YBRhkqaIx7aYx1F87E8n
From (redirected): https://drive.google.com/uc?id=1EY52bIpxMdf7YBRhkqaIx7aYx1F87E8n&confirm=t&uuid=fffa64ca-2cf4-48a0-bb04-294d6b1b31c5
To: /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/warp/test/checkpoints/warp_viton.pth
100% 117M/117M [00:04<00:00, 23.5MB/s]


In [27]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/warp/test/

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/warp/test


In [ ]:
!python -u eval_PBAFN_viton.py --name=cloth-warp --resize_or_crop=none --batchSize=32 --gpu_ids=0 --warp_checkpoint=checkpoints/warp_viton.pth --label_nc=13 --dataroot=/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/test_dataset --fineSize=512 --unpaired

Inference

In [31]:
!pip install omegaconf einops pandas scipy scikit-image matplotlib clip pytorch-lightning==1.6.5 lightning-utilities==0.4.0 taming-transformers-rom1504 transformers kornia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 971.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchmetrics to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 29.1 MB/s e

In [32]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/models/vgg

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/models/vgg


In [33]:
!gdown 1rvow8jStPt8t2prDcSRlnf8yzXhrYeGo

Downloading...
From (original): https://drive.google.com/uc?id=1rvow8jStPt8t2prDcSRlnf8yzXhrYeGo
From (redirected): https://drive.google.com/uc?id=1rvow8jStPt8t2prDcSRlnf8yzXhrYeGo&confirm=t&uuid=3046ccee-8bab-405f-b523-b61cd38a168a
To: /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/models/vgg/vgg19_conv.pth
100% 160M/160M [00:02<00:00, 59.0MB/s]


In [34]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/models/

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On/models


In [35]:
!gdown 177zXMarfVU6K_F-vtuC5fawNKThOu3ai

Downloading...
From (original): https://drive.google.com/uc?id=177zXMarfVU6K_F-vtuC5fawNKThOu3ai
From (redirected): https://drive.google.com/uc?id=177zXMarfVU6K_F-vtuC5fawNKThOu3ai&confirm=t&uuid=890cebac-22d4-4d89-a215-3450742ac614
To: /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/models/viton512_v2.ckpt
100% 5.32G/5.32G [01:06<00:00, 79.7MB/s]


In [36]:
%cd /content/drive/MyDrive/DCI-VTON-Virtual-Try-On/

/content/drive/MyDrive/DCI-VTON-Virtual-Try-On


In [38]:
!python test.py --plms --gpu_id 0 --ddim_steps 100 --outdir results/viton \
--config configs/viton512_v2.yaml \
--dataroot test_dataset \
--ckpt models/viton512_v2.ckpt \
--n_samples 4 \
--seed 23 \
--scale 1 \
--H 512 \
--W 512

2024-03-01 05:02:54.473517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 05:02:54.473579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 05:02:54.481741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 05:02:56.778503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Global seed set to 23
Loading model from models/viton512_v2.ckpt
Global Step: 9720
LatentTryOnDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.54 M params.
making attention of type 'vanilla' with 512 in_channels
Working wit

Image_resizing